In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import xgboost as xgb
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

In [35]:
class LeachateDataProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.df = None
        self.rock_features = [
            'EC_rock', 'Ph_rock', 'Corg_rock (%)', 'Ca_rock', 'K_rock',
            'Mg_rock', 'Na_rock', 'SAR_rock', 'SiO2_rock', 'Al2O3_rock',
            'Fe2O3_rock', 'TiO2_rock', 'MnO_rock', 'CaO_rock', 'MgO_rock',
            'Na2O_rock', 'K2O_rock', 'SO3_rock', 'P2O5_rock'
        ]
        self.env_features = ['Type_event', 'Event_quantity', 'Acid', 'Temp', 'Timestep']
        self.targets = [
            'Volume_leachate', 'EC_leachate', 'Ph_leachate', 'Chloride_leachate',
            'Carbonate_leachate', 'Sulfate_leachate', 'Nitrate_leachate',
            'Phosphate_leachate', 'Ca_leachate', 'Fe_leachate', 'K_leachate',
            'Mg_leachate', 'Mn_leachate', 'Na_leachate'
        ]

    def load_and_clean(self):
        # Load data with semicolon delimiter
        self.df = pd.read_csv(self.file_path, delimiter=';')

        # 1. Replace European decimals (,) with dots (.)
        # 2. Replace missing values '/' with NaN, then fill with 0 (assuming below detection limit)
        for col in self.df.columns:
            if self.df[col].dtype == 'object':
                self.df[col] = self.df[col].astype(str).str.replace(',', '.')
                self.df[col] = self.df[col].replace('/', 0)

                # Convert to numeric if possible (ignore 'Type_event' for now)
                if col != 'Type_event':
                    self.df[col] = pd.to_numeric(self.df[col], errors='coerce').fillna(0)

        # Encode Categorical Variables (Rain/Snow)
        self.df['is_rain'] = (self.df['Type_event'] == 'rain').astype(int)

        return self.df

    def engineer_features(self):
        """
        Crucial Step: Weathering is cumulative.
        We must calculate the total exposure the rock has had up to this timestep.
        """
        # Sort by Rock and Time to ensure cumulative sums are correct
        self.df = self.df.sort_values(by=['Rock_number', 'Timestep'])

        # Group by Rock Number
        g = self.df.groupby('Rock_number')

        # 1. Cumulative Water Exposure
        self.df['cum_water'] = g['Event_quantity'].cumsum()

        # 2. Cumulative Acid Load (Event Amount * Acid Concentration approximation)
        # Assuming 'Acid' column is a concentration factor, if it's pH, logic differs slightly.
        # Here we treat it as a load factor.
        self.df['cum_acid_load'] = g.apply(lambda x: (x['Event_quantity'] * x['Acid']).cumsum()).reset_index(level=0, drop=True)

        # 3. Time Variance (Lag features - what happened in the previous step?)
        # Shift targets to use previous leaching as input for current leaching
        # (Optional: Can improve accuracy but requires sequential prediction in deployment)
        # for target in self.targets:
        #     self.df[f'prev_{target}'] = g[target].shift(1).fillna(0)

        return self.df

In [36]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

class LeachatePredictor:
    def __init__(self, df, feature_cols, target_cols):
        self.df = df
        self.feature_cols = feature_cols
        self.target_cols = target_cols
        # Changed: We now store a dictionary of models, one per target
        self.models = {}
        self.scaler = StandardScaler()

    def prepare_data(self):
        X = self.df[self.feature_cols]
        y = self.df[self.target_cols]

        # Random split (or GroupShuffleSplit if using Rock_number)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        return X_train, X_test, y_train, y_test

    def train(self, X_train, y_train):
        print(f"Training separate models for {len(self.target_cols)} targets...")

        # Iterate through each specific leachate target
        for target in self.target_cols:
            # 1. Create a mask where the value is NOT 0
            # (Assuming 0 implies non-detect/missing)
            mask = y_train[target] != 0

            # 2. Filter X and y to exclude 0s for this specific target
            X_filtered = X_train.loc[mask]
            y_filtered = y_train.loc[mask, target]

            if len(X_filtered) < 10:
                print(f"  Skipping {target}: Not enough non-zero samples ({len(X_filtered)})")
                continue

            # 3. Define the Estimator
            model = xgb.XGBRegressor(
                booster="dart",
                objective='reg:squarederror',
                colsample_bytree=0.7,
                subsample=0.8,
                gamma=0,
                reg_alpha=0.1,
                reg_lambda=1,
                n_estimators=500,
                learning_rate=0.05,
                max_depth=6,
                random_state=42,
                n_jobs=-1
            )

            # 4. Fit only on non-zero data
            model.fit(X_filtered, y_filtered)

            # 5. Store the model
            self.models[target] = model

        print("Training Complete.")

    def evaluate(self, X_test, y_test):
        results = {}
        # We need a place to store predictions aligned with the original dataframe shape
        # Initialize with 0s
        all_preds_df = pd.DataFrame(0.0, index=X_test.index, columns=self.target_cols)

        for target in self.target_cols:
            if target not in self.models:
                continue # Skip if we didn't train a model for this (e.g. all 0s)

            model = self.models[target]

            # Generate predictions for ALL test rows (the model can predict non-zero even if actual is 0)
            preds = model.predict(X_test)
            all_preds_df[target] = preds

            # --- EVALUATION LOGIC ---
            # Mask the TEST set: Only score against actual non-zero values
            mask = y_test[target] != 0

            if mask.sum() == 0:
                results[target] = {'R2': np.nan, 'RMSE': np.nan}
                continue

            # Filter actuals and predictions using the mask
            y_true_filtered = y_test.loc[mask, target]
            y_pred_filtered = preds[mask] # preds is a numpy array, boolean indexing works if shape matches

            # Calculate metrics on non-zero subset
            r2 = r2_score(y_true_filtered, y_pred_filtered)
            rmse = np.sqrt(mean_squared_error(y_true_filtered, y_pred_filtered))

            results[target] = {'R2': r2, 'RMSE': rmse}

        return results, all_preds_df

In [42]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_results(y_test, preds_df, target_name):
    # 1. Filter Data: Only plot points where Actual value is NOT 0
    # This matches the training logic (ignoring non-detects)
    mask = y_test[target_name] != 0

    if mask.sum() < 5:
        print(f"Skipping plot for {target_name}: Not enough non-zero data points.")
        return

    actuals = y_test.loc[mask, target_name]
    predictions = preds_df.loc[mask, target_name]

    plt.figure(figsize=(8, 6))
    plt.scatter(actuals, predictions, alpha=0.6, color='blue')

    # Perfect prediction line
    min_val = min(actuals.min(), predictions.min())
    max_val = max(actuals.max(), predictions.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')

    plt.title(f'Actual vs Predicted: {target_name} (Non-Zero Only)')
    plt.xlabel('Actual Concentration/Value')
    plt.ylabel('Predicted Concentration/Value')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# --- 1. Plot Prediction Scatter Plots ---
# 'predictions' variable comes from: metrics, predictions = predictor.evaluate(X_test, y_test)
targets_to_plot = predictor.target_cols

for t in targets_to_plot:
    if t in y_test.columns and t in predictions.columns:
        plot_results(y_test, predictions, t)

# --- 2. Plot Feature Importance ---
# We now have multiple models (one per target).
# Let's plot importance for the first available target as an example.

if len(predictor.models) > 0:
    # Get the first target name and its corresponding model
    first_target_name = list(predictor.models.keys())[0]
    first_model = predictor.models[first_target_name]

    # Extract importances
    importances = pd.Series(first_model.feature_importances_, index=input_features).sort_values(ascending=False).head(10)

    plt.figure(figsize=(10, 6))
    importances.plot(kind='barh', color='teal')
    plt.title(f'Top 10 Feature Importance for {first_target_name}')
    plt.gca().invert_yaxis() # Highest importance at top
    plt.show()
else:
    print("No models were trained successfully.")

AttributeError: module 'sklearn.ensemble._hist_gradient_boosting.predictor' has no attribute 'target_cols'

In [30]:
# Save the trained model for future use
import joblib
joblib.dump(predictor.model, 'leachate_model.pkl')
print("Trained model saved as 'leachate_model.pkl'.")

Trained model saved as 'leachate_model.pkl'.
